# Mutation Pipeline

In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import gzip
import sys
sys.path.insert(0, '..')
from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from IPython.display import Image,display
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
from collections import OrderedDict
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [ ]:
samplesetname = "20Q2"
prevname="20Q1"
prevprevname ='19Q4'
prevprevversion= 15
virtual_public='public-20q1-c3b6'
virtual_dmc='dmc-20q1-c1d7'
virtual_internal='internal-20q1-f1a0'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"
source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
release = samplesetname

In [ ]:
%%R
release <- '20Q2'
prevname <- '20Q1'
genome_version <- 'hg19'
taiga_version <- 10

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
extract_to_change = {'from_arxspan_id': 'participant'}

## Adding new data

We are looking for new samples in a range of workspaces.

They are quite messy and might contains duplicates, contain broken file paths...

- We are thus looking at the bam files one by one and comparing them with our own bams. 
- We remove broken files, duplicates and add new version of a cell line's bam if we find some.

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

In [ ]:
# we will be missing "primary disease","sm_id", "cellosaurus_id", "gender, "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "comments"
#when SMid: match== 
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='wes', refurl=refsheet_url, wmfroms = [workspace1, workspace2, workspace3, workspace6], sources=[source1, source2, source3, source6], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

In [ ]:
# I am trying to remove duplicates from samples without arxspan ids to then look more into them and see if I have to get data for them or if I should just throw them out
toremov=set()
for k, val in noarxspan.iterrows():
    withsamesize = noarxspan[noarxspan["sample_id"] == val["sample_id"]]
    if len(withsamesize) > 1:
        for l, v in withsamesize.iloc[1:].iterrows():
            toremov.add(l)

In [ ]:
for i in toremov:
    noarxspan = noarxspan.drop(i)

In [ ]:
noarxspan

In [ ]:
noarxspan.sample_id = [i.split('_Exom')[0] for i in noarxspan.sample_id]

In [ ]:
len(noarxspan)

In [ ]:
noarxspan['ccle_name'] = [''.join(i.split('_')[1:-1]).split('_v')[0] for i in noarxspan.sample_id]
noarxspan['readgroup'] = [i.split('_')[0] for i in noarxspan.sample_id]

In [ ]:
for i, v in noarxspan.iterrows():
    if not gcp.exists(v['cram_or_bam_path']):
        print(v.ccle_name)
        noarxspan = noarxspan.drop(i)

In [ ]:
noarxspan['ccle_name'].tolist()

In [ ]:
len(noarxspan)

In [ ]:
toupdate = {"gender":[],
"primary_disease":[],
"sm_id":[],
"cellosaurus_id":[],
"age":[],
"primary_site":[],
"subtype":[],
"subsubtype":[],
"origin":[],
"comments":[],
"patient_id":[]}

In [ ]:
samples

In [ ]:
# If I have a previous samples I can update unknown data directly
index=[]
notfound=[]
for k, val in samples.iterrows():
    dat = ccle_refsamples[ccle_refsamples['arxspan_id']==val['arxspan_id']]
    if len(dat)>0:
        index.append(k)
        for k, v in toupdate.items():
            toupdate[k].append(dat[k].tolist()[0])
    else:
        notfound.append(k)

In [ ]:
len(index)

In [ ]:
# doing so..
for k, v in toupdate.items():
    samples.loc[index,k] =v

In [ ]:
len(samples.loc[notfound].patient_id)

In [ ]:
samples.loc[notfound].patient_id.tolist()

In [ ]:
# for these samples I will need to check and manually add the data in the list 
samples.loc[notfound]

In [ ]:
# found same patient
a = ["ACH-000635","ACH-000717", "ACH-000864", "ACH-001042", "ACH-001547"]
b = ["ACH-002291","ACH-001672"]

In [ ]:
ccle_refsamples[ccle_refsamples.arxspan_id.isin(a)].patient_id

In [ ]:
# duplicate ach-id
dup = {"ACH-001620": "ACH-001605",
"ACH-001621": "ACH-001606"}

In [ ]:
samples = changeCellLineNameInNewSet(new = samples, ref=ccle_refsamples, datatype="rna", dupdict=dup)

In [ ]:
#rename ccle_name TODO: ask becky what to do
rename = {"PEDS117": "CCLFPEDS0009T"}

In [ ]:
len(notfound)

## getting the addtional data and writing it here in the right order 'as shown above'
- use the stripped_cell_line_name to find the samples on https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE/edit#gid=356471436. 
- Make sure that we don't have duplicate cell lines in there. Otherwise, use the duplicate renaming function
- copy Primary Site, Primary Disease, Subtype, Comments, Disease Sub-subtype, if they exist. (sometimes subtype and subsubtype are the same.. don't use subsubtype then.
- look for the cell line in cellosaurus, you might need to use one of the aliases given in master depmap pv..
- copy  cellosaurus_id gender age info or write 'U' if they don't exist. 'can be a number or {Embryonic, Children, Adult, Fetus, U} 
- check that it does not say this cell line is not a duplicate from another cell line
- check that if it says this cell line is derived/children/father/samepatient from other cell lines, and that if we have any of the other cell lines, that the patient id is changed to be the same one for all (be sure that you are updating everywhere these patient ids are used)

In [ ]:
toupdate = {"gender":["Female","Female","Female"],
"primary_disease":["Breast Cancer","Breast Cancer","Breast Cancer"],
"cellosaurus_id":["CVCL_7932","CVCL_7931","CVCL_7933"],
"age":[37,37,37],
"primary_site":["pleural_effusion","pleural_effusion","breast"],
"subtype":["Carcinoma","Carcinoma","Carcinoma"],
"subsubtype":["","",""],
"comments":["HER2+; Received from Academic lab (Polyak, DFCI)","HER2+; Received from Academic lab (Polyak, DFCI)","HER2+; Received from Academic lab (Polyak, DFCI)"],
"stripped_cell_line_name":["21MT2","21MT1", "21NT"],
"patient_id":['PT-y3RbI7uD', 'PT-y3RbI7uD', 'PT-y3RbI7uD']}

In [ ]:
a  = pd.DataFrame(toupdate)
a['name'] = samples.loc[notfound,"stripped_cell_line_name"].tolist()
a

In [ ]:
# updating..
for k, v in toupdate.items():
    samples.loc[notfound,k] =v

In [ ]:
# uploading to our bucket (now a new function)
samples = h.changeToBucket(samples,'gs://cclebams/wes/', values=['internal_bam_filepath','internal_bai_filepath'], catchdup=False)

In [ ]:
samples

In [ ]:
# saving and updating the spreadsheet with these
print("YOU NOW NEED TO UPDATE THE GOOGLE SHEET!")
samples.to_csv('temp/new_ccle_samples.csv')

In [ ]:
samples['arxspan_id'].tolist()

In [ ]:
samples

In [ ]:
samples = samples.rename(columns={'patient_id':'participant_id'})

In [ ]:
pairs

In [ ]:
pairs = pairs.rename(columns={'patient_id':'participant_id'}).set_index('pair_id')

In [ ]:
pairs.participant_id = samples.participant_id.tolist()

In [ ]:
sam = cnwm.get_samples()
sam[sam['baits']=="AGILENT"]

In [ ]:
#uploading new samples to mut
refwm = refwm.disable_hound()
refwm.upload_samples(samples)
refwm.upload_entities('pairs', pairs)
refwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = refwm.get_samples()

pair = refwm.get_pairs()
refwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
refwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
refwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
refwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
refwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
refwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

In [ ]:
#and CN
cnwm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_CN_hg38')
cnwm = cnwm.disable_hound()
cnwm.upload_samples(samples)
cnwm.upload_entities('pairs', pairs)
cnwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = cnwm.get_samples()

pair = cnwm.get_pairs()
cnwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
cnwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
cnwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
#cnwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
#cnwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
#cnwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
cnwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run Terra pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the mutation dataset:

*   For new samples in DepMap, run the ICE version of this task. CCLE2 samples used Agilent targets, so this pipeline should be used instead. The pipelines are identical in terms of their outputs, but the proper targets, baits, and pseudo normal should be used based on how the samples were sequenced.

    **ICE_CGA_Production_Analysis_Pipeline_Cell_Lines_copy** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22) OR


    **AGILENT_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Anablysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22)

*   **common_variant_filter** (breardon/common_variant_filterSnapshot ID: 3)
*   **filterMAF_on_CGA_pipeline** (gkugener/filterMAF_on_CGA_pipelineSnapshot ID: 8)
*   **aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1)

This outputs to be downloaded will be saved in the sample set that was run. The output we use for the release is:


*   **passedCGA_filteredMAF_aggregated** 

There are several other tasks in this workspace. In brief:



*   **CGA_Production_Analysis_Pipeline_Cell_Lines** (lelagina/CGA_Production_Analysis_Pipeline_Cell_LinesSnapshot ID: 12). This task is the same as the ICE and AGILENT prefixed version above, except that it relied on pulling the baits and targets to use from the metadata stored for the samples. Having AGILENT and ICE versions specified made the uploading and running process easier.
*   **SANGER_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22). This task was trying to run the CGA pipeline on the Sanger WES data, using a Sanger pseudo normal. In its current implementation, this task fails to complete for the samples.
*   **UNFILTERED_aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1). Aggregates the MAF outputted by the CGA cell line pipeline prior to the common variant filter and germline filtering tasks. This can give us insight to which mutations are getting filtered out when. We may want to potentially include this MAF in the release so people can see why certain mutations of interest may be getting filtered out.
*   WES_DM_Mutation_Calling_Pipeline_(standard |expensive) (gkugener/WES_DM_Mutation_Calling_PipelineSnapshot ID: 2). This was a previous mutation calling pipeline implemented for CCLE. We do not use this pipeline any more as the CGA pipeline looks better.
*   aggregate_filterMAF_CGA (CCLE/aggregate_filterMAF_CGASnapshot ID: 1). An aggregation MAF task that we used in the past. We do not use this task anymore.
*   calculate_mutational_burden (breardon/calculate_mutational_burdenSnapshot ID: 21). This task can be used to calculate the mutational rate of the samples. We do not make use of this data in the release although it could be of interest.
*   summarizeWigFile (breardon/summarizeWigFileSnapshot ID: 5). CCLF ran this task (might be necessary for the mutational burden task). For our workflow, we do not run it.

## On Terra

In [ ]:
submission_id = refwm.create_submission("CGA_WES_CCLE_ICE", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id = refwm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id = refwm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

### filtered

In [ ]:
submission_id1 = refwm.create_submission("aggregateMAFs_selectFields_copy", samplesetname)

### unfiltered

In [ ]:
submission_id2 = refwm.create_submission("aggregateMAFs_selectFields_unfiltered", samplesetname)

In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1,submission_id2])

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'./data/'+samplesetname+'/Mutconfig')

## On local

### downloading from terra

In [ ]:
res = refwm.get_sample_sets().loc[samplesetname]
res

In [ ]:
filtered = res['filtered_CGA_MAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

###### retrieving RNAseq MAFs

### postprocessing


Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

- readMutations
- createSNPs
- addToMainMutation
- filterAllelicFraction
- filterMinCoverage
- mergeAnnotations
- addAnnotation
- maf_add_variant_annotations
- mutation_maf_to_binary_matrix (x3)

In [ ]:
a = pd.read_csv('temp/mutation_filtered_terra_merged.txt',sep='\t')

In [ ]:
a.Tumor_Sample_Barcode

In [ ]:
a.Tumor_Sample_Barcode [i[0] for i in a.Tumor_Sample_Barcode]

In [ ]:
a

In [ ]:
a.to_csv('temp/mutation_filtered_terra_merged.txt',sep='\t')

In [ ]:
%%R
source('../gkugener/RScripts/load_libraries_and_annotations.R')
load('../gkugener/DM_Omics/Annotations.rdata') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)
source('src/CCLE_postp_function.R')
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(cdsomics)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed

In [ ]:
%%R
newly_merged_maf <- readMutations('temp/q3_mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
names(new_release)

In [ ]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe', data.file=paste0('mutations.','19Q2'),data.version=3)
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == "") {
 previous.release.maf[,1] <- NULL 
}
prevnames <- names(previous.release.maf)
prevnames

In [ ]:
%%R
print(nrow(previous.release.maf))
merged <- addToMainMutation(previous.release.maf, new_release)
print(nrow(merged))

In [ ]:
%%R
## Adding more
newly_merged_maf <- readMutations('temp/mutation_filtered_terra_merged.txt')
new_release <- createSNPs(newly_merged_maf)
print(names(new_release))
merged <- addToMainMutation(merged, new_release)
nrow(merged)

In [ ]:
%%R
setdiff(names(merged), names(previous.release.maf))

In [ ]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

In [ ]:
%%R
dim(merged)

In [ ]:
%%R
filtered <- filterAllelicFraction(merged)

In [ ]:
%%R
head(new_release[which(new_release$Tumor_Sample_Barcode=='ACH-002466'),])

In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
head(merged)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
# Guillaume's version
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))
# Allie's version
new_release <- maf_add_variant_annotations(new_release)

In [ ]:
%%R
# some matric files that does get used internaly and might be useful
damaging_mutation <- mutation_maf_to_binary_matrix(new_release, damaging =  TRUE)
other_mutation <- mutation_maf_to_binary_matrix(new_release, other = TRUE)
hotspot_mutation <- mutation_maf_to_binary_matrix(new_release, hotspot = TRUE)

In [ ]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)
# Save the ready to upload file to upload to taiga
write.table(
  damaging_mutation, 
  paste0('temp/damaging_mutation.', release, '.tsv'), sep = '\t', quote = F)
# Save the ready to upload file to upload to taiga
write.table(
  other_mutation, 
  paste0('temp/other_mutation.', release, '.tsv'), sep = '\t', quote = F)
# Save the ready to upload file to upload to taiga
write.table(
  hotspot_mutation, 
  paste0('temp/hotspot_mutation.', release, '.tsv'), sep = '\t', quote = F)

# Validation

## Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [ ]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')
damaging_mutation = pd.read_csv('temp/damaging_mutation.'+release+'.tsv', sep='\t')
print(len(damaging_mutation))
other_mutation = pd.read_csv('temp/other_mutation.'+release+'.tsv', sep='\t')
print(len(other_mutation))
hotspot_mutation = pd.read_csv('temp/hotspot_mutation.'+release+'.tsv', sep='\t')
print(len(hotspot_mutation))

In [ ]:
b = ['ACH-001441',
'ACH-001453',
'ACH-002024',
'ACH-002026',
'ACH-002041',
'ACH-002038',
'ACH-002046',
'ACH-001528',
'ACH-001370',
'ACH-001627',
'ACH-002039',
'ACH-001648',
'ACH-001649',
'ACH-001653',
'ACH-002062',
'ACH-001398',
'ACH-002025',
'ACH-001688',
'ACH-001692',
'ACH-001694',
'ACH-001229',
'ACH-001670',
'ACH-001498',
'ACH-001961',
'ACH-002476',
'ACH-002475',
'ACH-002045',
'ACH-002044',
'ACH-002066',
'ACH-001807',
'ACH-002510',
'ACH-002511',
'ACH-001819',
'ACH-001577',
'ACH-001993',
'ACH-001613',
'ACH-001617',
'ACH-002017',
'ACH-002059',
'ACH-001172',
'ACH-002466',
'ACH-002465',
'ACH-002462',
'ACH-002467',
'ACH-002464',
'ACH-002463',
'ACH-002029',
'ACH-001201',
'ACH-001699',
'ACH-002508',
'ACH-002509']
[i for i in b if i not in set(mutations.DepMap_ID)]

In [ ]:
len(mutations.columns)

In [ ]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831]

In [ ]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [ ]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [ ]:
ac_data.shape[0]

## check important mutations

In [ ]:
# check MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# check TP53 mutation 

In [ ]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(str(100*pos/ac_data.shape[0]),end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

In [ ]:
allnan

### basic counts

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)

In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)

QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

In [ ]:
prevprev= set(tc.get(name='depmap-mutation-calls-9be3', file="depmap_"+prevprevname+"_mutation_calls", version=prevprevversion).DepMap_ID.tolist())

# uploading on taiga

In [ ]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if i is not np.nan]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if i is not np.nan]

In [ ]:
wes_dmc_embargo

In [ ]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

In [ ]:
! cd ../depmap-release-readmes && git pull

In [ ]:
!cd ../depmap-release-readmes/ && python make_new_release.py 20Q1 && git add . && git commit -m "20Q1" && git push 

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+releAse+'/internal-'+releAse+'.txt ../ccle_processing/temp/README && cd -')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'}, 
                 dataset_description="""
# Mutations

filtered and unfiltered mutation files from Broad WES and Sanger WES data mapped to hg19
The MAF file for DepMap that includes all of the latest WES samples. This MAF is generated by merging CCLE (WGS, RNAseq, RD, HC) and Sanger (WES) data.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1:  In 19Q1 the WES_AC column has been replaced by two columns, VA_WES_AC and CGA_WES_AC. We are currently using the Van Allen and CGA based pipeline to generate mutation calls. The CGA pipeline includes more filtering on the MAFs than VA and has a better INDEL caller. However, some of these filters may be removing some variants of interest that are still capture by the VA pipeline, which is why both a retained for now. DEPRECATED:  Missing the VA_WES_AC, CGA_WES_AC columns
version 2: 19Q1 data
version 3: 19Q2 data. We are no longer using the CCLE_WES_AC column. We are only using the CGA pipeline for mutation calls.
version 4: Updating to 19Q3interim DEPRECATED
version 5: Updating to 19Q3interim DEPRECATED
version 6: Updating to 19Q3interim
version 7: Updating to 19Q3 DEPRECATED
version 8: reparing the missing mutation problem DEPRECATED
version 9: reparing the missing column problem

## ** Internal 19Q4****

version10:
Adding 52 new cell lines. 
Some cells lines have been flagged as:

version11:
adding missing cell lines

Adding 52 new cell lines. 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - Genes having a similar CN value accross all []


genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)

## Internal

In [ ]:
print(len(mutations))
mutations = mutations[~mutations.DepMap_ID.isin(blacklist)]
print(len(mutations))
mutations.to_csv('temp/depmap_'+release+'_mutation_calls', sep='\t', index=False)
print(len(damaging_mutation))
damaging_mutation = damaging_mutation[~damaging_mutation.index.isin(blacklist)]
print(len(damaging_mutation))
damaging_mutation.to_csv('temp/damaging_mutation', sep='\t', index=False)
print(len(other_mutation))
other_mutation = other_mutation[~other_mutation.index.isin(blacklist)]
print(len(other_mutation))
other_mutation.to_csv('temp/other_mutation', sep='\t', index=False)
print(len(hotspot_mutation))
hotspot_mutation = hotspot_mutation[~hotspot_mutation.index.isin(blacklist)]
print(len(hotspot_mutation))
hotspot_mutation.to_csv('temp/hotspot_mutation', sep='\t', index=False)

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-9be3",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'NumericMatrixTSV',
                                        'temp/other_mutation': 'NumericMatrixTSV',
                                        'temp/hotspot_mutation': 'NumericMatrixTSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description="""
# Internal Mutations

Mutation calls for Internal DepMap data

* Version 1 Internal 18Q1*

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18Q1_maf_20180202.txt`
* Version 2-4 Internal 18Q2*

merged mutations and indels file (1,606 cell lines, including CCLE and Sanger WES reanalysis)
original source: 
`/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q2_maf_20180502.txt`
Binary matrices:
- damaging: if isDeleterious is true
- missense: if isDeleterious is false
- hotspot: if missense and either TCGA or COSMIC hotspot
Version 2 contains the MAF file
* Version 5-6 Internal 18Q3*

version 5 deprecated

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_depMap_18q3_maf_20180716.txt`

Binary matrices:
- damaging: if isDeleterious is true
- missense: if isDeleterious is false
- hotspot: if missense and either TCGA or COSMIC hotspot
- Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 7-8 Internal 18Q4*

version 8 just changes a column name in the MAF file from Broad_ID to DepMap_ID

original source: `/xchip/ccle_dist/broad_only/CMAG/mutations/CCLE_DepMap_18Q4_maf_20181028.txt`

* Version 9-12 Internal 19Q1*

version 12 updates the column name from VA_WES_AC to CCLE_WES_AC

version 11+ uses an updated definition for hotspot mutations

version 12 contains the correct data for 19Q1

* Version 13 Internal 19Q2*

* Version 14-15 Internal 19Q3*

version 15 fixed entrez ids

* Version 16 Internal 19Q4*

adding 35 new cell lines.

* Version 16 Internal 19Q4*
uploading as matrices

* Version 17 Internal 19Q4*
removing unauthorized lines and setting as matrices

*** Variant annotation column ***

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
""")

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-mutation-calls-9be3', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls')])

## DMC

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+releAse+'/dmc-'+releAse+'.txt ../ccle_processing/temp/README && cd -')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-dfce",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'NumericMatrixTSV',
                                        'temp/other_mutation': 'NumericMatrixTSV',
                                        'temp/hotspot_mutation': 'NumericMatrixTSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description="""
# DMC Mutations

* Version 1-5 DMC 19Q1*

version 5 is a one-off portal thing because dmc wanted to be able to plot if a gene has any mutation as one-hot encoded value in the x/y axes of the data explorer It adds the any_mutation matrix, but does not change the others. Code used to generate:

```
from taigapy import TaigaClient

c = TaigaClient()

dmc_19q1_mutation_taiga_root = "depmap-mutation-calls-dfce.3/"
other_matrix = c.get(dmc_19q1_mutation_taiga_root + "other_mutation")
damaging_matrix = c.get(dmc_19q1_mutation_taiga_root + "damaging_mutation")
hotspot_matrix = c.get(dmc_19q1_mutation_taiga_root + "hotspot_mutation")

df = other_matrix.append(damaging_matrix)
df = df.groupby(level=0).sum()

df = df.append(hotspot_matrix)
df = df.groupby(level=0).sum()

df[df > 1] = 1

df.to_csv('any_mutation.csv')
```
The code uses version 3 because the dmc portal was using version 3

version 4 updates the column name from VA_WES_AC to CCLE_WES_AC

version 3 has an updated definition for hotspot mutations

version 2+ contains the correct data for 19Q1

* Version 6 DMC 19Q2*

* Version 7-8 DMC 19Q3*
version 8 fixed entrez ids

* Version 9 DMC 19Q4*
adding 52 new cell lines.

* Version 10 DMC 19Q4*
removing unauthorized lines and setting as matrices

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:
- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
 """)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_dmc, 'depmap-mutation-calls-dfce', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])

## Public

In [ ]:
print(len(mutations))
mutations = mutations[mutations.DepMap_ID.isin(prevprev)]
print(len(mutations))
mutations = mutations[~mutations.DepMap_ID.isin(wes_dmc_embargo)]
print(len(mutations))
mutations.to_csv('temp/depmap_'+release+'_mutation_calls', sep='\t', index=False)
print(len(damaging_mutation))
damaging_mutation = damaging_mutation[damaging_mutation.index.isin(prevprev)]
print(len(damaging_mutation))
damaging_mutation = damaging_mutation[~damaging_mutation.index.isin(wes_dmc_embargo)]
print(len(damaging_mutation))
damaging_mutation.to_csv('temp/damaging_mutation', sep='\t', index=False)
print(len(other_mutation))
other_mutation = other_mutation[other_mutation.index.isin(prevprev)]
print(len(other_mutation))
other_mutation = other_mutation[~other_mutation.index.isin(wes_dmc_embargo)]
print(len(other_mutation))
other_mutation.to_csv('temp/other_mutation', sep='\t', index=False)
print(len(hotspot_mutation))
hotspot_mutation = hotspot_mutation[hotspot_mutation.index.isin(prevprev)]
print(len(hotspot_mutation))
hotspot_mutation = hotspot_mutation[~hotspot_mutation.index.isin(wes_dmc_embargo)]
print(len(hotspot_mutation))
hotspot_mutation.to_csv('temp/hotspot_mutation', sep='\t', index=False)

In [ ]:
os.system('cd ../depmap-release-readmes && git pull && mv release-'+releAse+'/public-'+releAse+'.txt README && cd -')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutation-calls-9a1a",
                 upload_file_path_dict={'temp/depmap_'+release+'_mutation_calls': 'TableTSV',
                                        'temp/damaging_mutation': 'NumericMatrixTSV',
                                        'temp/other_mutation': 'NumericMatrixTSV',
                                        'temp/hotspot_mutation': 'NumericMatrixTSV',
                                       },#'temp/README': 'Raw'},
                 dataset_description="""
# Public Mutations

Mutation calls for Public DepMap data

* Version 1 Public 18Q1*

original source: CCLE data portal
* Version 2 Public 18Q2*

merged mutations and indels file (1,549 cell lines total, including data for 63 newly released cell lines)
original source: `/xchip/ccle_dist/public/DepMap_18Q2/CCLE_DepMap_18Q2_maf_20180502.txt`
* Version 3-4 Public 18Q3*

version 3 deprecated

original source: `/xchip/ccle_dist/public/DepMap_18Q3/CCLE_DepMap_18q3_maf_20180718.txt`

Binary matrices:
damaging: if isDeleterious is true
missense: if isDeleterious is false
hotspot: if missense and either TCGA or COSMIC hotspot
Rows: cell line, Broad (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)

MAF file

* Version 5 Public 18Q4*

original source: `/xchip/ccle_dist/public/DepMap_18Q4/CCLE_DepMap_18q4_maf_20181029.txt`

* Version 6-9 Public 19Q1*

version 9 updates the column name from VA_WES_AC to CCLE_WES_AC

version 8 uses an updated definition for hotspot mutations

version 9 contains the correct data for 19Q1

* Version 10 Public 19Q2*

* Version 11-12 Public 19Q3*

version 12 fixed entrez ids

* Version 13 Public 19Q4*

adding 52 new cell lines

* Version 14 Public 19Q4*
removing unauthorized lines and setting matrices

MAF file, added column (Variant_annotation) classifying each variant as either silent, damaging, other conserving, or other non-conserving, based on this mapping (old annotation from Variant_Classification column - new annotation):

Silent - silent
Splice_Site - damaging
Missense_Mutation - other non-conserving
Nonsense_Mutation - damaging
De_novo_Start_OutOfFrame - damaging
Nonstop_Mutation - other non-conserving
Frame_Shift_Del - damaging
Frame_Shift_Ins - damaging
In_Frame_Del - other non-conserving
In_Frame_Ins - other non-conserving
Stop_Codon_Del - other non-conserving
Stop_Codon_Ins - other non-conserving
Start_Codon_SNP - damaging
Start_Codon_Del - damaging
Start_Codon_Ins - damaging
5'Flank - other conserving
Intron - other conserving
IGR - other conserving
3'UTR - other conserving
5'UTR - other conserving
Binary matrices:

- damaging: if damaging
- other: if other conserving or other non-conserving
- hotspot: if it is not a silent mutation and is either TCGA or COSMIC hotspot
- Rows: cell line, DepMap (arxspan) IDs

Columns: Gene, HGNC symbol (Entrez ID)
 """)

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_public, 'depmap-mutation-calls-9a1a', [('CCLE_mutations', 'depmap_'+release+'_mutation_calls'),])#('README','README')])